# Initial Lean Verifier Training

### Step 1: Download Model
We discussed 2 candidate models:
1. [LeanDojo's ReProver*](https://github.com/lean-dojo/ReProver?tab=readme-ov-file#using-trained-models-on-hugging-face)
2. [Llemma (LM for Math)](https://huggingface.co/EleutherAI/llemma_7b)

*we probably want the Lean 4 + Retiever-Less version: [hf card](https://huggingface.co/kaiyuy/leandojo-lean4-tacgen-byt5-small)

ReProver is trained specially on Lean data, while Llemma is trained on various math texts, including proof assistant code (Lean, Isabelle).
ReProver however is a ByT5-small derivative, so it only has 300M parameters. 
In comparison, Llemma is 7B model (based on CodeLlama).

In the long term we probably want a richer 7B model to optimize against.
But for initial experiments I'll start with ReProver for efficiency.

In [1]:
import json
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import perf_counter
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM, # Llemma
    AutoModelForSeq2SeqLM,
)

/Users/matthewho/miniconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("hello world")

hello world


In [3]:
# this is the retriever-less version
# - can switch "lean4 <-> lean3"
reprover_model_path = "kaiyuy/leandojo-lean4-tacgen-byt5-small"
reprover_tokenizer = AutoTokenizer.from_pretrained(reprover_model_path)
reprover = AutoModelForSeq2SeqLM.from_pretrained(reprover_model_path)

tokenizer_config.json: 100%|██████████| 25.6k/25.6k [00:00<00:00, 17.9MB/s]
added_tokens.json: 100%|██████████| 3.02k/3.02k [00:00<00:00, 30.1MB/s]
special_tokens_map.json: 100%|██████████| 3.09k/3.09k [00:00<00:00, 31.5MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100%|██████████| 837/837 [00:00<00:00, 7.91MB/s]
model.safetensors: 100%|██████████| 1.20G/1.20G [00:27<00:00, 43.6MB/s]
generation_config.json: 100%|██████████| 142/142 [00:00<00:00, 1.63MB/s]


Tactic code example...

In [ ]:
state = "n : ℕ\n⊢ gcd n n = n"
tokenized_state = reprover_tokenizer(state, return_tensors="pt")

# Generate a single tactic.
tactic_ids = reprover.generate(tokenized_state.input_ids, max_length=1024)
tactic = reprover_tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
print(tactic, end="\n\n")

# Generate multiple tactics via beam search.
# add timing code to get a feel for latency
_start = perf_counter()
tactic_candidates_ids = reprover.generate(
    tokenized_state.input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False,
)
print(f"Generated 4 candidates via beam search in {perf_counter() - _start}s")
tactic_candidates = reprover_tokenizer.batch_decode(
    tactic_candidates_ids, skip_special_tokens=True
)
for tac in tactic_candidates:
    print(tac)

## Step 2: Prep Data
LeanDojo's (lean4) dataset is hosted [here](https://zenodo.org/records/10114185).
The tar file is also copied in this repo (`REPO_ROOT/data/leandojo_benchmark_4_v5.tar.gz`)

In [ ]:
# utility
REPO_ROOT = "/mnt/hdd/msho/gfn_ntp/"
def full_path(path_relative_to_project_repo_root):
    return os.path.join(REPO_ROOT, path_relative_to_project_repo_root)

In [ ]:
# let's unpack the tar file
import tarfile
data_dir = full_path("data/")
tar_path = os.path.join(REPO_ROOT, "data/leandojo_benchmark_4_v5.tar.gz")
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(path=data_dir)